### Data preprocessing

In [3]:
#import necessary libraries and files 
import pandas as pd
import numpy as np
import warnings
import time
import datetime as dt
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import folium
import sklearn
import seaborn as sns

from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans


%matplotlib inline

In [4]:
data1 = pd.read_csv('dublinbikes_20190401_20190701.csv')
data2 = pd.read_csv('dublinbikes_20190701_20191001.csv')
data3 = pd.read_csv('dublinbikes_20191001_20200101.csv')
data4 = pd.read_csv('dublinbikes_20200101_20200401.csv')
data5 = pd.read_csv('dublinbikes_20180701_20181001.csv')
data6 = pd.read_csv('dublinbikes_20180701_20181001.csv')
data1.sample(5)

,STATION ID,TIME,LAST UPDATED,NAME,BIKE STANDS,AVAILABLE BIKE STANDS,AVAILABLE BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE
574188,76,2019-04-18 02:25:03,2019-04-18 02:15:51,MARKET STREET SOUTH,38,16,22,Open,Market Street South,53.342297,-6.287661
556813,14,2019-04-18 18:30:04,2019-04-18 18:29:01,FOWNES STREET UPPER,30,3,27,Open,Fownes Street Upper,53.344604,-6.263371
2138496,85,2019-06-05 16:30:02,2019-06-05 16:21:57,ROTHE ABBEY,35,25,10,Open,Rothe Abbey,53.338776,-6.303950
336134,40,2019-04-11 12:35:05,2019-04-11 12:32:16,JERVIS STREET,21,0,21,Open,Jervis Street,53.348301,-6.266651
106844,35,2019-04-04 09:05:03,2019-04-04 09:03:50,SMITHFIELD,30,26,4,Open,Smithfield,53.347691,-6.278214


In [7]:
#merge data into one table
data = pd.concat([data1,data2, data3,data4])
data = data[data['STATUS'] == 'Open']
data = data[(data['LAST UPDATED'] >= '2019-07-01') & (data['LAST UPDATED'] < '2020-04-01')]
data = data[(data['LAST UPDATED'] < '2019-12-01') | (data['LAST UPDATED'] >= '2020-02-01')]

#remove rows where no update actually occurs
data = data.drop(['TIME'], axis = 1)
data.drop_duplicates(keep= 'first',inplace=True)


#get date and time columns
data['DATETIME'] = [dt.datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in data["LAST UPDATED"]]
data['LAST UPDATED'] = [dt.datetime.time(d) for d in data['DATETIME']] 
data['DATE'] = [dt.datetime.date(d) for d in data['DATETIME']] 
data['date_for_merge'] = data['DATETIME'].dt.round('H')

#create important features
data['OCCUPANCY_PCT'] =  data['AVAILABLE BIKES'] / data['BIKE STANDS']
data['FULL'] = np.where(data['OCCUPANCY_PCT'] == 0, 1,0 )
data['EMPTY'] = np.where(data['OCCUPANCY_PCT'] == 1, 1,0 )

### create time aggregates needed for clustering
# weekday/saturday/sunday
data['DAY_NUMBER'] = data.DATETIME.dt.dayofweek
data['DAY_TYPE'] = np.where(data['DAY_NUMBER'] <= 4, 'Weekday', (np.where(data['DAY_NUMBER'] == 5, 'Saturday', 'Sunday')))

def bin_time(x):
    if x.time() < dt.time(6):
        return "Overnight "
    elif x.time() < dt.time(11):
        return "6AM-10AM "
    elif x.time() < dt.time(16):
        return "11AM-3PM "
    elif x.time() < dt.time(20):
        return "4PM-7PM "
    elif x.time() <= dt.time(23):
        return "8PM-11PM "
    else:
        return "Overnight "


data["TIME_TYPE"] = data['DATETIME'].apply(bin_time)
data['HOUR'] = data['DATETIME'].dt.hour
data['MONTH'] = data['DATETIME'].dt.month
data['CLUSTER_GROUP'] = data['TIME_TYPE'] + data['DAY_TYPE']

data.sample(5)

,STATION ID,LAST UPDATED,NAME,BIKE STANDS,AVAILABLE BIKE STANDS,AVAILABLE BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE,...,date_for_merge,OCCUPANCY_PCT,FULL,EMPTY,DAY_NUMBER,DAY_TYPE,TIME_TYPE,HOUR,MONTH,CLUSTER_GROUP
72423,30,11:10:46,PARNELL SQUARE NORTH,20,19,1,Open,Parnell Square North,53.353462,-6.265305,...,2019-07-03 11:00:00,0.050,0,0,2,Weekday,11AM-3PM,11,7,11AM-3PM Weekday
1441150,107,00:33:45,CHARLEVILLE ROAD,40,22,18,Open,Charleville Road,53.359158,-6.281866,...,2019-08-14 01:00:00,0.450,0,0,2,Weekday,Overnight,0,8,Overnight Weekday
1633846,69,09:14:38,GRAND CANAL DOCK,40,12,27,Open,Grand Canal Dock,53.342636,-6.238695,...,2019-11-21 09:00:00,0.675,0,0,3,Weekday,6AM-10AM,9,11,6AM-10AM Weekday
1843984,61,18:03:31,HARDWICKE PLACE,25,20,5,Open,Hardwicke Place,53.357044,-6.263232,...,2019-08-27 18:00:00,0.200,0,0,1,Weekday,4PM-7PM,18,8,4PM-7PM Weekday
1581291,93,21:29:02,HEUSTON STATION (CENTRAL),40,0,40,Open,Heuston Station (Central),53.346603,-6.296924,...,2019-11-19 21:00:00,1.000,0,1,1,Weekday,8PM-11PM,21,11,8PM-11PM Weekday


In [8]:
data.head()

,STATION ID,LAST UPDATED,NAME,BIKE STANDS,AVAILABLE BIKE STANDS,AVAILABLE BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE,...,date_for_merge,OCCUPANCY_PCT,FULL,EMPTY,DAY_NUMBER,DAY_TYPE,TIME_TYPE,HOUR,MONTH,CLUSTER_GROUP
1,2,00:02:22,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.35677,-6.26814,...,2019-07-01 00:00:00,0.1,0,0,0,Weekday,Overnight,0,7,Overnight Weekday
3,2,00:12:31,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.35677,-6.26814,...,2019-07-01 00:00:00,0.1,0,0,0,Weekday,Overnight,0,7,Overnight Weekday
5,2,00:22:41,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.35677,-6.26814,...,2019-07-01 00:00:00,0.1,0,0,0,Weekday,Overnight,0,7,Overnight Weekday
7,2,00:32:50,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.35677,-6.26814,...,2019-07-01 01:00:00,0.1,0,0,0,Weekday,Overnight,0,7,Overnight Weekday
9,2,00:42:59,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.35677,-6.26814,...,2019-07-01 01:00:00,0.1,0,0,0,Weekday,Overnight,0,7,Overnight Weekday


### 1. Find missing data in the dataframes.

In [9]:
# Returns the number of rows and columns
# 4 million rows, 22 columns
data.shape

(4154564, 22)

In [10]:
# returns True values where there is a NaN or Null value
data.isnull()

,STATION ID,LAST UPDATED,NAME,BIKE STANDS,AVAILABLE BIKE STANDS,AVAILABLE BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE,...,date_for_merge,OCCUPANCY_PCT,FULL,EMPTY,DAY_NUMBER,DAY_TYPE,TIME_TYPE,HOUR,MONTH,CLUSTER_GROUP
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2196589,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2196591,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2196593,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2196595,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
# Shows how many NaN values there are
# There appear to be no NaN values, therefore the data must be missing
# in some other way
data.isnull().sum()

STATION ID               0
LAST UPDATED             0
NAME                     0
BIKE STANDS              0
AVAILABLE BIKE STANDS    0
AVAILABLE BIKES          0
STATUS                   0
ADDRESS                  0
LATITUDE                 0
LONGITUDE                0
DATETIME                 0
DATE                     0
date_for_merge           0
OCCUPANCY_PCT            0
FULL                     0
EMPTY                    0
DAY_NUMBER               0
DAY_TYPE                 0
TIME_TYPE                0
HOUR                     0
MONTH                    0
CLUSTER_GROUP            0
dtype: int64

In [13]:
data

,STATION ID,LAST UPDATED,NAME,BIKE STANDS,AVAILABLE BIKE STANDS,AVAILABLE BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE,...,date_for_merge,OCCUPANCY_PCT,FULL,EMPTY,DAY_NUMBER,DAY_TYPE,TIME_TYPE,HOUR,MONTH,CLUSTER_GROUP
1,2,00:02:22,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.356770,-6.268140,...,2019-07-01 00:00:00,0.1,0,0,0,Weekday,Overnight,0,7,Overnight Weekday
3,2,00:12:31,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.356770,-6.268140,...,2019-07-01 00:00:00,0.1,0,0,0,Weekday,Overnight,0,7,Overnight Weekday
5,2,00:22:41,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.356770,-6.268140,...,2019-07-01 00:00:00,0.1,0,0,0,Weekday,Overnight,0,7,Overnight Weekday
7,2,00:32:50,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.356770,-6.268140,...,2019-07-01 01:00:00,0.1,0,0,0,Weekday,Overnight,0,7,Overnight Weekday
9,2,00:42:59,BLESSINGTON STREET,20,18,2,Open,Blessington Street,53.356770,-6.268140,...,2019-07-01 01:00:00,0.1,0,0,0,Weekday,Overnight,0,7,Overnight Weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2196589,117,23:12:21,HANOVER QUAY EAST,40,40,0,Open,Hanover Quay East,53.343655,-6.231755,...,2020-03-31 23:00:00,0.0,1,0,1,Weekday,Overnight,23,3,Overnight Weekday
2196591,117,23:22:30,HANOVER QUAY EAST,40,40,0,Open,Hanover Quay East,53.343655,-6.231755,...,2020-03-31 23:00:00,0.0,1,0,1,Weekday,Overnight,23,3,Overnight Weekday
2196593,117,23:32:39,HANOVER QUAY EAST,40,40,0,Open,Hanover Quay East,53.343655,-6.231755,...,2020-04-01 00:00:00,0.0,1,0,1,Weekday,Overnight,23,3,Overnight Weekday
2196595,117,23:42:48,HANOVER QUAY EAST,40,40,0,Open,Hanover Quay East,53.343655,-6.231755,...,2020-04-01 00:00:00,0.0,1,0,1,Weekday,Overnight,23,3,Overnight Weekday


This prints the amount of unique values there are for the date_for_merge column. This gives us 24 values for each day in the dataset, the amount of hours in each day. This can be used to find out if there is any missing data.

In [23]:
print(data['date_for_merge'].unique())

['2019-07-01T00:00:00.000000000' '2019-07-01T01:00:00.000000000'
 '2019-07-01T02:00:00.000000000' ... '2020-03-31T22:00:00.000000000'
 '2020-03-31T23:00:00.000000000' '2020-04-01T00:00:00.000000000']


In [14]:
#July, August, September, October, November, December, January,
# February, March, April 1st 00:00:00 (1 value)

# If we multiply the amount of days in each month by 24 hours,
# we should find out how many date_for_merge entries there should be.

1+(31+31+30+31+30+31+31+28+31)*24

#6577

6577

In [22]:
# Prints how many date_for_merge entries there actually are.

print(data['date_for_merge'].nunique())

5099


I suspected some months were missing, so I decided to see what percentage of entries are missing - around 77% are present of what there should be. 

In [15]:
5099/6577

0.7752774821347119

...which is around the same percentage as if two months were missing.

In [27]:
7/9

0.7777777777777778

I temporarily changed the date_for_merge column to string type to make it easier to see what was missing. 

In [18]:
data['date_for_merge'] = data['date_for_merge'].astype(str)

In [19]:
type(data['date_for_merge'][1])

str

I checked each month to see if any data were missing for the first entry of each month.
January came up as false, indicating it's first entry was missing.

This led me to suspect that the issue was with the month of January.

In [20]:
print("June '19")
print('2019-06-01 00:00:00' in data['date_for_merge'].unique())
print("July '19")
print('2019-07-01 00:00:00' in data['date_for_merge'].unique())
print("August '19")
print('2019-08-01 00:00:00' in data['date_for_merge'].unique())
print("September '19")
print('2019-09-01 00:00:00' in data['date_for_merge'].unique())
print("October '19")
print('2019-10-01 00:00:00' in data['date_for_merge'].unique())
print("November '19")
print('2019-11-01 00:00:00' in data['date_for_merge'].unique())
print("December '19")
print('2019-12-01 00:00:00' in data['date_for_merge'].unique())
print("Jan '20")
print('2020-01-01 00:00:00' in data['date_for_merge'].unique())
print("Feb '20")
print('2020-02-01 00:00:00' in data['date_for_merge'].unique())
print("March '20")
print('2020-03-01 00:00:00' in data['date_for_merge'].unique())
print("April '20")
print('2020-04-01 00:00:00' in data['date_for_merge'].unique())
print("May '20")
print('2020-05-01 00:00:00' in data['date_for_merge'].unique())


June '19
False
July '19
True
August '19
True
September '19
True
October '19
True
November '19
True
December '19
True
Jan '20
False
Feb '20
True
March '20
True
April '20
True
May '20
False


In [85]:
print('2019-12-01 00:00:00' in data['date_for_merge'].unique())

True


In [87]:
print('2019-12-01 01:00:00' in data['date_for_merge'].unique())

False


In [16]:
print('2019-12-15 01:00:00' in data['date_for_merge'].unique())

False


In [88]:
print('2020-01-01 00:00:00' in data['date_for_merge'].unique())

False


In [93]:
print('2020-01-31 23:00:00' in data['date_for_merge'].unique())

False


In [92]:
print('2020-02-01 00:00:00' in data['date_for_merge'].unique())

True
